# (IGNORE) ST_3: EDA on Newly Read File

In [ ]:
pt_semiclean_2020_v0.head(5)

In [ ]:
pt_semiclean_2020_v0.shape

In [ ]:
pt_semiclean_2020_v0.index

In [ ]:
pt_semiclean_2020_v0.tail(10)

In [ ]:
pt_semiclean_2020_v0.isnull()

In [ ]:
pt_semiclean_2020_v0.index.nunique()

In [ ]:
pt_semiclean_2020_v0.dtypes

In [ ]:
#pt_semiclean_2020_v0_zc_astime = pt_semiclean_2020_v0.astype({'zip_code':'float64'})

In [ ]:
pt_semiclean_2020_v1_dtindex = pt_semiclean_2020_v0.set_index(['date_time'])

In [ ]:
pt_semiclean_2020_v0.head(2)

In [ ]:
pt_semiclean_2020_v1_dtindex.head(2)

In [ ]:
pt_semiclean_2020_v1_dtindex.index

In [ ]:
pt_semiclean_2020_v1_dtindex.tail(5)

# ST_1: Install and import time-series packages and modules

In [ ]:
#!pip install chart_studio

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import seaborn as sns #extra
import graphviz #extra
import pydot #extra
import copy #extra
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') #special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline
from pylab import rcParams
from chart_studio import plotly #new -> import chart_studio for plotly package
from plotly import tools
#import plotly.plotly as py 
#Note that plotly.plotly is deprecated
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
from numpy.random import normal, seed
from scipy.stats import norm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARIMA
import math
from sklearn.metrics import mean_squared_error
#print(os.listdir("../input"))
#print(os..) causes error


# ST_2: Read in CSV File

In [2]:
pd.options.display.float_format = '{:,.2f}'.format #only concern 2 digits after the decimal point
pd.set_option('display.width', 75) #safer option than .set_option.display.width bc regex will match to multiple option names
pd.set_option('display.max_columns', 10)

In [3]:
#this file does not have leap year values
#REMINDER: CHECK IF THE PARSE DATES FUNCTION WORKED OR NOT
pt_semiclean2020_dec_to_april = pd.read_csv('/content/drive/MyDrive/Coding_projects/Chang-COVID/pt_semiclean_2020_dec10_april21.csv',\
                              names=['date_time','visit_id','mrn','age','gender','race','ethnicity',\
                                     'eddispo','acuity_level',\
                                     'dx_cancer','dx_hiv','dx_asthma','dx_copd','dx_cad','dx_diab','dx_liver','dx_chf','dx_ckd','dx_renalflr','dx_htn',\
                                     'zip_code'],\
                              #index_col='date_time',\
                              skiprows=1,\
                              parse_dates=['date_time'],
                              low_memory=False)

In [5]:
type(pt_semiclean2020_dec_to_april)

pandas.core.frame.DataFrame

In [9]:
pt_semiclean2020_dec_to_april.isnull().sum()

date_time        0
visit_id         0
mrn              0
age              0
gender           0
race             1
ethnicity       65
eddispo          0
acuity_level     0
dx_cancer        0
dx_hiv           0
dx_asthma        0
dx_copd          0
dx_cad           0
dx_diab          0
dx_liver         0
dx_chf           0
dx_ckd           0
dx_renalflr      0
dx_htn           0
zip_code         0
dtype: int64

In [7]:
pt_semiclean2020_dec_to_april[pt_semiclean2020_dec_to_april.isnull().any(axis=1)]

,date_time,visit_id,mrn,age,gender,...,dx_chf,dx_ckd,dx_renalflr,dx_htn,zip_code
1400,2019-12-12 00:54:00,384871081,2680089,48,0,...,0,0,0,0,75116
2459,2019-12-13 15:26:00,384931494,1376558,46,0,...,0,0,0,0,75216
2462,2019-12-13 15:37:00,384932031,901240,37,0,...,0,0,0,0,75241
3188,2019-12-14 18:12:00,384943138,1798781,24,1,...,0,0,0,0,75227
7242,2019-12-20 20:43:00,385122421,1632853,26,0,...,0,0,0,0,75229
...,...,...,...,...,...,...,...,...,...,...,...
73545,2020-04-03 17:24:00,387807147,2746677,60,0,...,0,0,0,0,75050
74254,2020-04-06 00:17:00,387818435,1958921,57,0,...,0,0,0,0,76051
74436,2020-04-06 12:47:00,387836474,788797,60,0,...,0,0,0,0,75114
74572,2020-04-06 18:48:00,387849568,5003736,30,0,...,0,0,0,1,75201


In [10]:
pt_semiclean2020_april_to_nov = pd.read_csv('/content/drive/MyDrive/Coding_projects/Chang-COVID/pt_semiclean_2020_april22_to_nov9.csv',\
                              names=['date_time','visit_id','mrn','age','gender','race','ethnicity',\
                                     'eddispo','acuity_level',\
                                     'dx_cancer','dx_hiv','dx_asthma','dx_copd','dx_cad','dx_diab','dx_liver','dx_chf','dx_ckd','dx_renalflr','dx_htn',\
                                     'zip_code'],\
                              #index_col='date_time',\
                              skiprows=1,\
                              parse_dates=['date_time'],
                              low_memory=False)

In [11]:
pt_semiclean2020_april_to_nov.head(2)

,date_time,visit_id,mrn,age,gender,...,dx_chf,dx_ckd,dx_renalflr,dx_htn,zip_code
0,2020-04-22 11:14:00,"388,216,817.00","4,465,064.00",46.00,1.00,...,0.00,0.00,0.00,0.00,75041
1,2020-04-22 11:17:00,"388,216,942.00","286,188.00",55.00,0.00,...,0.00,0.00,0.00,1.00,75241


In [12]:
pt_semiclean2020_april_to_nov.isnull().sum()

date_time       945770
visit_id        945770
mrn             945770
age             945770
gender          945770
race            945770
ethnicity       945862
eddispo         945770
acuity_level    945770
dx_cancer       945770
dx_hiv          945770
dx_asthma       945770
dx_copd         945770
dx_cad          945770
dx_diab         945770
dx_liver        945770
dx_chf          945770
dx_ckd          945770
dx_renalflr     945770
dx_htn          945770
zip_code        945774
dtype: int64

In [13]:
pt_semiclean2020_april_to_nov.tail(5)

,date_time,visit_id,mrn,age,gender,...,dx_chf,dx_ckd,dx_renalflr,dx_htn,zip_code
1048051,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1048052,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1048053,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1048054,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
1048055,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN


In [14]:
pt_semiclean2020_april_to_nov_v1 = pd.read_csv('/content/drive/MyDrive/Coding_projects/Chang-COVID/pt_semiclean_2020_april22_to_nov9_v1.csv',\
                              names=['date_time','visit_id','mrn','age','gender','race','ethnicity',\
                                     'eddispo','acuity_level',\
                                     'dx_cancer','dx_hiv','dx_asthma','dx_copd','dx_cad','dx_diab','dx_liver','dx_chf','dx_ckd','dx_renalflr','dx_htn',\
                                     'zip_code'],\
                              #index_col='date_time',\
                              skiprows=1,\
                              parse_dates=['date_time'],
                              low_memory=False)

In [15]:
pt_semiclean2020_april_to_nov_v1.isnull().sum()

date_time        0
visit_id         0
mrn              0
age              0
gender           0
race             0
ethnicity       92
eddispo          0
acuity_level     0
dx_cancer        0
dx_hiv           0
dx_asthma        0
dx_copd          0
dx_cad           0
dx_diab          0
dx_liver         0
dx_chf           0
dx_ckd           0
dx_renalflr      0
dx_htn           0
zip_code         4
dtype: int64

In [16]:
pt_semiclean2020_april_to_nov_v1[pt_semiclean2020_april_to_nov_v1['zip_code'].isnull()]

,date_time,visit_id,mrn,age,gender,...,dx_chf,dx_ckd,dx_renalflr,dx_htn,zip_code
63092,2020-08-30 11:55:00,391813188,3085031,40,1,...,0,0,0,1,NaN
73167,2020-09-17 18:56:00,392345244,3085031,40,1,...,0,0,0,1,NaN
73931,2020-09-19 07:42:00,392382569,3085031,40,1,...,0,0,0,1,NaN
93140,2020-10-23 10:23:00,393399356,3085031,40,1,...,0,0,0,1,NaN
